In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:////tmp/mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
client.list_experiments()

In [ ]:
client.create_experiment(name="my-cool-experiment")

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5001")

In [ ]:
run_id="013ed19c4b7449e492fbdd2f8492137d"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(
    model_uri=model_uri,
    name="just_test"
)

In [ ]:
client.list_registered_models()

In [ ]:
model_name="just_test"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
new_stage="Staging"
client.transition_model_version_stage(
    name = model_name,
    version=2,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime
model_version=2
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"the model version {model_version} was transitioned to {new_stage} on {date}"
)

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import mlflow 
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <=60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + "_" + df["DOLocationID"]
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models://{name}/{stage}")
    y_pred = model.predict(X_test)
    return {
        "rmse" : mean_squared_error(y_test, y_pred, squared=False)
    }

In [ ]:
df = read_dataframe('/Users/xxx/mlops-zoomcamp/02-experiment-tracking_hw/data/green_tripdata_2021-03.parquet')

In [ ]:

run_id="120a4af1ce7a4d3a93e008675c08e2fd"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

import pickle
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values
model_name="test/1"

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [ ]:
mlflow.pyfunc.load_model('runs:/120a4af1ce7a4d3a93e008675c08e2fd/models_mlflow')